In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from sklearn.metrics import classification_report, confusion_matrix

EMBEDDING_DIM = 256
HIDDEN_DIM = 512
DROPOUT = 0.4
BATCH_SIZE = 64
EPOCHS = 10

MAX_SEQUENCE_LENGTH = 250
MAX_TOKENS = 20000

try:
    df = pd.read_csv("amazon_reviews.csv")
except FileNotFoundError:
    print("="*50)
    print("="*50)
    df = pd.DataFrame(columns=["reviewText", "overall"])

if not df.empty:
    df = df[["reviewText", "overall"]].dropna()
    df = df[df["overall"] != 3]
    df["label"] = (df["overall"] >= 4).astype(int)

    pos_df = df[df["label"] == 1]
    neg_df = df[df["label"] == 0]

    neg_df_upsampled = resample(
        neg_df,
        replace=True,
        n_samples=len(pos_df),
        random_state=42
    )

    df_balanced = pd.concat([pos_df, neg_df_upsampled]).sample(frac=1, random_state=42).reset_index(drop=True)

    print("Before upsampling:")
    print(df["label"].value_counts())
    print("\nAfter upsampling:")
    print(df_balanced["label"].value_counts())

    train_texts, test_texts, train_labels, test_labels = train_test_split(
        df_balanced["reviewText"], df_balanced["label"].values,
        test_size=0.2,
        random_state=42
    )

    vectorize_layer = layers.TextVectorization(
        max_tokens=MAX_TOKENS,
        output_mode='int',
        output_sequence_length=MAX_SEQUENCE_LENGTH
    )


    print("\nAdapting TextVectorization layer...")
    vectorize_layer.adapt(train_texts)

    VOCAB_SIZE = vectorize_layer.vocabulary_size()
    print(f"Vocabulary size: {VOCAB_SIZE}")

    def vectorize_text(text, label):
        text = tf.expand_dims(text, -1)
        return vectorize_layer(text), label

    train_ds = tf.data.Dataset.from_tensor_slices((train_texts, train_labels))
    train_ds = train_ds.batch(BATCH_SIZE).map(vectorize_text).prefetch(tf.data.AUTOTUNE)

    test_ds = tf.data.Dataset.from_tensor_slices((test_texts, test_labels))
    test_ds = test_ds.batch(BATCH_SIZE).map(vectorize_text).prefetch(tf.data.AUTOTUNE)

    embedding_layer = layers.Embedding(VOCAB_SIZE, EMBEDDING_DIM, mask_zero=True)

    model = keras.Sequential([

        layers.Input(shape=(MAX_SEQUENCE_LENGTH,), dtype="int64"),

        embedding_layer,

        layers.Dropout(DROPOUT),

        layers.Bidirectional(
            layers.LSTM(HIDDEN_DIM, return_sequences=True)
        ),

        layers.Dropout(DROPOUT),

        layers.Bidirectional(
            layers.LSTM(HIDDEN_DIM, return_sequences=False)
        ),

        layers.Dropout(DROPOUT),

        layers.Dense(1, activation='sigmoid')
    ])

    model.summary()

    model.compile(
        loss='binary_crossentropy',
        optimizer=keras.optimizers.Adam(
            learning_rate=0.001,
            clipnorm=5.0
        ),
        metrics=['accuracy']
    )

    print("\n--- Starting Training ---")
    history = model.fit(
        train_ds,
        epochs=EPOCHS,
        validation_data=test_ds
    )

    print("\n--- Evaluating Model ---")
    loss, accuracy = model.evaluate(test_ds)
    print(f"\n Test Accuracy (Balanced via Upsampling): {accuracy:.4f}")

    all_preds_prob = model.predict(test_ds)
    all_preds = (all_preds_prob > 0.5).astype(int).squeeze()

    all_labels = np.concatenate([labels for _, labels in test_ds], axis=0)

    print("\n Classification Report:")
    print(classification_report(all_labels, all_preds, target_names=["Negative", "Positive"], digits=4))

    print("\n Confusion Matrix:")
    print(confusion_matrix(all_labels, all_preds))

    def predict_sentiment(text):
        encoded_text = vectorize_layer([text])

        prediction = model.predict(encoded_text, verbose=0)
        pred_prob = prediction[0][0]

        return "Positive" if pred_prob > 0.5 else "Negative"

    positive_texts = [
        "I really love this product!",
        "Absolutely fantastic, exceeded my expectations.",
        "This is the best purchase I've made this year.",
        "High quality and works perfectly.",
        "I am very satisfied with this item.",
        "Excellent product, would buy again.",
        "Totally worth the money, highly recommend.",
        "Amazing! Fast shipping and great quality.",
        "Very happy with this, five stars!",
        "This product is incredible, love it!"
    ]

    negative_texts = [
        "Terrible quality, very disappointed.",
        "Broke after one use, waste of money.",
        "Do not buy this, completely useless.",
        "Poorly made and arrived late.",
        "Not as described, extremely unhappy.",
        "This product is awful, avoid it.",
        "Very disappointed, will never buy again.",
        "Cheap material, doesn't work at all.",
        "Horrible experience, one star.",
        "Regret buying this, total waste."
    ]

    print("\n--- Positive Sentiment Predictions ---")
    for text in positive_texts:
        sentiment = predict_sentiment(text)
        print(f"'{text}' -> {sentiment}")

    print("\n--- Negative Sentiment Predictions ---")
    for text in negative_texts:
        sentiment = predict_sentiment(text)
        print(f"'{text}' -> {sentiment}")

Before upsampling:
label
1    4448
0     324
Name: count, dtype: int64

After upsampling:
label
0    4448
1    4448
Name: count, dtype: int64

Adapting TextVectorization layer...
Vocabulary size: 9719


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 250, 256)       │     2,488,064 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 250, 256)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 250, 1024)      │     3,149,824 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 250, 1024)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_1 (Bidirectional) │ (None, 1024)           │     6,295,552 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │         1,025 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,934,465 (45.53 MB)

 Trainable params: 11,934,465 (45.53 MB)

 Non-trainable params: 0 (0.00 B)


--- 🚀 Starting Training ---
Epoch 1/10
112/112 ━━━━━━━━━━━━━━━━━━━━ 27s 173ms/step - accuracy: 0.8020 - loss: 0.4682 - val_accuracy: 0.9624 - val_loss: 0.1150
Epoch 2/10
112/112 ━━━━━━━━━━━━━━━━━━━━ 19s 172ms/step - accuracy: 0.9707 - loss: 0.0904 - val_accuracy: 0.9685 - val_loss: 0.1166
Epoch 3/10
112/112 ━━━━━━━━━━━━━━━━━━━━ 19s 173ms/step - accuracy: 0.9625 - loss: 0.1303 - val_accuracy: 0.9820 - val_loss: 0.0943
Epoch 4/10
112/112 ━━━━━━━━━━━━━━━━━━━━ 19s 170ms/step - accuracy: 0.9924 - loss: 0.0410 - val_accuracy: 0.9792 - val_loss: 0.0555
Epoch 5/10
112/112 ━━━━━━━━━━━━━━━━━━━━ 19s 169ms/step - accuracy: 0.9930 - loss: 0.0286 - val_accuracy: 0.9871 - val_loss: 0.0364
Epoch 6/10
112/112 ━━━━━━━━━━━━━━━━━━━━ 19s 169ms/step - accuracy: 0.9953 - loss: 0.0194 - val_accuracy: 0.9893 - val_loss: 0.0319
Epoch 7/10
112/112 ━━━━━━━━━━━━━━━━━━━━ 19s 171ms/step - accuracy: 0.9979 - loss: 0.0084 - val_accuracy: 0.9803 - val_loss: 0.0870
Epoch 8/10
112/112 ━━━━━━━━━━━━━━━━━━━━ 19s 172ms/step